In [17]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageFilter

def calculate_accuracy(detected_counts, actual_counts):
    if len(detected_counts) != len(actual_counts):
        raise ValueError("Les listes detected_counts et actual_counts doivent avoir la même longueur.")
    
    correct_detections = sum(1 for detected, actual in zip(detected_counts, actual_counts) if detected == actual)
    accuracy_rate = (correct_detections / len(actual_counts)) * 100
    
    return accuracy_rate

def open_specific_file():
    initial_dir = "/Users/chawkibhd/Desktop/dataset/1/coins_images"

    file_path = filedialog.askopenfilename(
        initialdir=initial_dir,
        title="Sélectionner le fichier image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.gif")]
    )
    
    if file_path:
        try:
            img = Image.open(file_path)
            base_width = 300
            w_percent = (base_width / float(img.size[0]))
            h_size = int((float(img.size[1]) * float(w_percent)))
            img = img.resize((base_width, h_size), Image.Resampling.LANCZOS)
            img = ImageTk.PhotoImage(img)

            if panel_original.winfo_exists():
                panel_original.config(image=img)
                panel_original.image = img

            global image_path
            image_path = file_path

        except Exception as e:
            messagebox.showerror("Erreur", f"Impossible de charger l'image : {e}")

def execute_processing():
    if not image_path:
        messagebox.showerror("Erreur", "Veuillez d'abord télécharger une image.")
        return

    try:
        img = Image.open(image_path)
        base_width = 300
        w_percent = (base_width / float(img.size[0]))
        h_size = int((float(img.size[1]) * float(w_percent)))
        img_resized = img.resize((base_width, h_size), Image.Resampling.LANCZOS)
        img_resized = ImageTk.PhotoImage(img_resized)

        img_processed = img.filter(ImageFilter.GaussianBlur(radius=5))
        img_processed = img_processed.resize((base_width, h_size), Image.Resampling.LANCZOS)
        img_processed = ImageTk.PhotoImage(img_processed)

        if panel_processed.winfo_exists():
            panel_processed.config(image=img_processed)
            panel_processed.image = img_processed

        taux_dexactitude = calculate_accuracy(detected_counts, actual_counts)

        footer_label_1.config(text=f"small = {small:.3f} // medium = {medium:.3f} // large = {large:.3f}")
        footer_label_2.config(text=f"taux d'exactitude = {taux_dexactitude:.3f}%")

    except Exception as e:
        messagebox.showerror("Erreur", f"Impossible de traiter l'image : {e}")

detected_counts = [10, 5, 7, 8, 6,7]
actual_counts = [10, 5, 7, 8, 6,3]

small, medium, large = 0, 0, 0
taux_dexactitude = 0
image_path = None

root = tk.Tk()
root.title("Coin counting")
root.geometry("800x500")
root.configure(bg="#f0f8ff")

title_label = tk.Label(root, text="Coin counting", font=("Helvetica", 20, "bold"), bg="#b03844", fg="white")
title_label.pack(fill="x", pady=15)

frame = tk.Frame(root, bg="#f0f8ff")
frame.pack(pady=20)

label_image = tk.Label(frame, text="Insérer votre image :", font=("Helvetica", 14), bg="#f0f8ff", fg="#4682B4")
label_image.grid(row=1, column=0, padx=10, pady=10, sticky="e")
button_uploader = tk.Button(frame, text="Uploader", command=open_specific_file, font=("Helvetica", 14), bg="#b58c1d", fg="white")
button_uploader.grid(row=1, column=1, padx=10, pady=10)

label_image = tk.Label(frame, text="Exécuter :", font=("Helvetica", 14), bg="#f0f8ff", fg="#4682B4")
label_image.grid(row=2, column=0, padx=10, pady=10, sticky="e")
button_execute = tk.Button(frame, text="Exécuter", command=execute_processing, font=("Helvetica", 14), bg="#1db58c", fg="white")
button_execute.grid(row=2, column=1, padx=10, pady=10)

footer_label_1 = tk.Label(root, text=f"small = {small:.3f} // "  f"medium = {medium:.3f} // " f"large = {large:.3f}", font=("Helvetica", 10), bg="#f0f8ff", fg="#4682B4")
footer_label_1.pack(pady=10)

footer_label_2 = tk.Label(root, text=f"taux d'exactitude = {taux_dexactitude:.3f}% " , font=("Helvetica", 10), bg="#f0f8ff", fg="#4682B4")
footer_label_2.pack(pady=15)

frame_image = tk.Frame(root, bg="#f0f8ff")
frame_image.pack(pady=20)

label_image_2 = tk.Label(frame_image, text="image d'origine", font=("Helvetica", 14), bg="#f0f8ff", fg="#4682B4")
label_image_2.grid(row=0, column=0, padx=10, pady=5)

panel_original = tk.Label(frame_image)
panel_original.grid(row=1, column=0, padx=10, pady=10)

label_image_3 = tk.Label(frame_image, text="image traitée", font=("Helvetica", 14), bg="#f0f8ff", fg="#4682B4")
label_image_3.grid(row=0, column=1, padx=10, pady=5)

panel_processed = tk.Label(frame_image)
panel_processed.grid(row=1, column=1, padx=10, pady=10)

footer_label_3 = tk.Label(root, text="Créé par chawki et akram", font=("Helvetica", 10), bg="#f0f8ff", fg="#4682B4")
footer_label_3.pack(side="bottom", pady=10)

image_path = None

root.mainloop()



In [ ]:
import pandas as pd
from os import path

def calculate_accuracy(img_path, detected_counts):
    # Check if the image path is valid
    if not path.exists(img_path):
        return "Error: Image path does not exist."

    # Ensure detected_counts is a valid number
    if not isinstance(detected_counts, (int, float)) or detected_counts < 0:
        return "Error: Detected counts must be a non-negative number."

    # Extract the image name from the path
    image_name = path.basename(img_path)

    # Load the dataset and handle potential errors
    try:
        dataset = pd.read_csv("../dataset/coins_count_values.csv")
    except FileNotFoundError:
        return "Error: Dataset file not found."
    except Exception as e:
        return f"Error: Failed to load dataset. Details: {e}"

    # Check if the image exists in the dataset
    result = dataset[dataset['image_name'] == image_name]
    if result.empty:
        return "Error: Image not found in the dataset."

    # Extract the actual coin count and calculate accuracy
    actual_counts = result.iloc[0]['coins_count']
    
    try:
        percentage = (detected_counts * 100) / actual_counts
    except ZeroDivisionError:
        return "Error: Actual counts in the dataset is zero, cannot calculate percentage."

    # Return the results
    return {
        "accuracy_percentage": percentage,
        "detected_counts": detected_counts,
        "actual_counts": actual_counts
    }

    


In [21]:
img_path = "../dataset/coins_images/coins_images/indian_coins/f3035e00dd.jpg"
detected_counts = 5

result = calculate_accuracy(img_path, detected_counts)
if isinstance(result, dict):
    print(f"Accuracy: {result['accuracy_percentage']}%")
    print(f"Detected Counts: {result['detected_counts']}")
    print(f"Actual Counts: {result['actual_counts']}")
else:
    print(result)  # Print the error message


Accuracy: 40.0%
Detected Counts: 5
Actual Counts: 2


In [16]:
import pandas as pd

def calculate_accuracy(detected_counts, img_path):
    # Vérifier que les deux listes ont la même longueur
    img_name = path.basename(img_path)
    dataset = pd.read_csv("../dataset/coins_count_values.csv")
    
    if len(detected_counts) != len(actual_counts):
        raise ValueError("Les listes detected_counts et actual_counts doivent avoir la même longueur.")
    
    # Calculer le nombre de détections correctes
    correct_detections = sum(1 for detected, actual in zip(detected_counts, actual_counts) if detected == actual)
    
    # Calculer le taux d'exactitude
    accuracy_rate = (correct_detections / len(actual_counts)) * 100
    
    return accuracy_rate

# Exemple d'utilisation
detected_counts = [10, 5, 7, 8, 6]  # Résultats de votre fonction
actual_counts = [10, 5, 7, 8, 6]    # Nombres réels de pièces

accuracy_rate = calculate_accuracy(detected_counts, actual_counts)
print(f"Taux d'exactitude: {accuracy_rate:.2f}%")


TypeError: expected str, bytes or os.PathLike object, not list